Dudas:
- Hay problema si a unos datos que me parece que ya estan escalados los vuelvo a escalar? o si yo mismo escalo dos veces?
- logistic como regression? check predict_proba

# Get and explore datasets

In [2]:
import pandas as pd
import numpy as np

 - Load the 2 databases: one with the diagnosis of the patients and other with the entire RNA expression levels for each gene in each patient

In [3]:
lung_diag = pd.read_csv("./data/2019 array/Lung/sample-data-relation_E-MTAB-6699.sdrf.txt", sep="\t",  header=0, index_col=0)

In [4]:
lung_rna = pd.read_csv("./data/2019 array/Lung/esets_lung_exprs_genes.txt", sep="\t",  header=0, index_col=0)

In [5]:
lung_diag = lung_diag.reset_index()

In [6]:
lung_diag.shape

(1621, 21)

In [7]:
lung_diag.columns

Index(['Source Name', 'Characteristics[organism]', 'Characteristics[disease]',
       'Characteristics[GEO accession number]', 'Material Type',
       'Protocol REF', 'Protocol REF.1', 'Extract Name', 'Protocol REF.2',
       'Labeled Extract Name', 'Label', 'Protocol REF.3', 'Assay Name',
       'Technology Type', 'Array Design REF', 'Term Source REF',
       'Protocol REF.4', 'Protocol REF.5', 'Derived Array Data File',
       'Comment [Derived ArrayExpress FTP file]', 'Factor Value[disease]'],
      dtype='object')

In [8]:
lung_diag.head(3)

,Source Name,Characteristics[organism],Characteristics[disease],Characteristics[GEO accession number],Material Type,Protocol REF,Protocol REF.1,Extract Name,Protocol REF.2,Labeled Extract Name,...,Protocol REF.3,Assay Name,Technology Type,Array Design REF,Term Source REF,Protocol REF.4,Protocol REF.5,Derived Array Data File,Comment [Derived ArrayExpress FTP file],Factor Value[disease]
0,GSM1019138,Homo sapiens,tumor,GSE37745,organism part,P-MTAB-74105,P-MTAB-74106,GSM1019138,P-MTAB-74107,GSM1019138:biotin,...,P-MTAB-74108,GSM1019138:biotin,array assay,A-AFFY-44,ArrayExpress,P-MTAB-74109,P-MTAB-74110,esets_lung_exprs_genes.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,tumor
1,GSM1019139,Homo sapiens,tumor,GSE37745,organism part,P-MTAB-74105,P-MTAB-74106,GSM1019139,P-MTAB-74107,GSM1019139:biotin,...,P-MTAB-74108,GSM1019139:biotin,array assay,A-AFFY-44,ArrayExpress,P-MTAB-74109,P-MTAB-74110,esets_lung_exprs_genes.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,tumor
2,GSM1019140,Homo sapiens,tumor,GSE37745,organism part,P-MTAB-74105,P-MTAB-74106,GSM1019140,P-MTAB-74107,GSM1019140:biotin,...,P-MTAB-74108,GSM1019140:biotin,array assay,A-AFFY-44,ArrayExpress,P-MTAB-74109,P-MTAB-74110,esets_lung_exprs_genes.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,tumor


In [9]:
lung_diag = lung_diag.rename(columns= {"Source Name":"patient", "Characteristics[disease]":"diagnosis"})

In [10]:
lung_diag["diagnosis"].value_counts()

tumor     1474
normal     147
Name: diagnosis, dtype: int64

In [11]:
lung_diag_sel = lung_diag[["patient","diagnosis"]]

In [12]:
lung_diag_sel.head()

,patient,diagnosis
0,GSM1019138,tumor
1,GSM1019139,tumor
2,GSM1019140,tumor
3,GSM1019141,tumor
4,GSM1019142,tumor


In [13]:
lung_rna.head(3)

,GSM258551.CEL,GSM258552.CEL,GSM258553.CEL,GSM258554.CEL,GSM258555.CEL,GSM258556.CEL,GSM258557.CEL,GSM258558.CEL,GSM258559.CEL,GSM258560.CEL,...,GSM748357.CEL,GSM748358.CEL,GSM748359.CEL,GSM748360.CEL,GSM748361.CEL,GSM748362.CEL,GSM748363.CEL,GSM748364.CEL,GSM748365.CEL,GSM748366.CEL
RFC2,7.427582,7.520136,8.607107,7.550951,8.748494,8.524921,7.757673,7.056589,8.124251,8.733339,...,7.604340,8.078034,8.743091,8.299523,8.632561,8.429557,8.735258,7.721628,8.716090,7.736672
HSPA6,6.548161,7.640517,7.538416,6.624467,7.324919,8.788338,9.951198,6.773493,9.159630,7.583808,...,7.950681,8.333995,8.512271,7.441800,7.125374,8.460541,8.210732,8.333649,7.968744,7.794173
PAX8,7.511132,7.114491,7.631883,8.097713,7.663455,7.291632,8.009362,7.956185,7.638113,7.568160,...,8.092287,7.557731,7.419319,7.508527,7.503761,7.494472,7.361972,7.478495,7.176204,7.970554


In [14]:
# transpose it to have the patients in the rows
lung_rna = lung_rna.transpose()

In [15]:
lung_rna.shape

(1621, 20545)

In [16]:
lung_rna.head(3)

,RFC2,HSPA6,PAX8,GUCA1A,THRA,PTPN21,CCL5,CYP2E1,EPHB3,ESRRA,...,NAF1,CCDC63,C16orf52,LOC101929475,LOC389641,LOC400499,GALR3,NUS1P3,TMEM231,LOC100505915
GSM258551.CEL,7.427582,6.548161,7.511132,5.897988,6.352771,6.316551,9.066054,4.984092,7.723588,7.691311,...,6.124568,4.456250,3.503017,4.176526,5.343218,6.784405,6.160561,4.188891,5.897904,5.936242
GSM258552.CEL,7.520136,7.640517,7.114491,5.362284,6.119202,7.606950,9.922118,6.259126,7.207981,7.466835,...,6.082873,4.616373,3.677266,4.405394,4.757592,6.401263,5.551504,4.505053,5.065979,5.663643
GSM258553.CEL,8.607107,7.538416,7.631883,6.015103,6.567469,4.968846,9.886879,4.773658,7.958417,7.293182,...,6.788019,5.006049,3.845536,4.448895,4.794517,6.048864,6.363465,4.626576,6.435507,5.151310


In [17]:
lung_rna = lung_rna.reset_index()

In [18]:
lung_rna.head(3)

,index,RFC2,HSPA6,PAX8,GUCA1A,THRA,PTPN21,CCL5,CYP2E1,EPHB3,...,NAF1,CCDC63,C16orf52,LOC101929475,LOC389641,LOC400499,GALR3,NUS1P3,TMEM231,LOC100505915
0,GSM258551.CEL,7.427582,6.548161,7.511132,5.897988,6.352771,6.316551,9.066054,4.984092,7.723588,...,6.124568,4.456250,3.503017,4.176526,5.343218,6.784405,6.160561,4.188891,5.897904,5.936242
1,GSM258552.CEL,7.520136,7.640517,7.114491,5.362284,6.119202,7.606950,9.922118,6.259126,7.207981,...,6.082873,4.616373,3.677266,4.405394,4.757592,6.401263,5.551504,4.505053,5.065979,5.663643
2,GSM258553.CEL,8.607107,7.538416,7.631883,6.015103,6.567469,4.968846,9.886879,4.773658,7.958417,...,6.788019,5.006049,3.845536,4.448895,4.794517,6.048864,6.363465,4.626576,6.435507,5.151310


In [19]:
lung_rna = lung_rna.rename(columns = {"index":"patient"})

In [20]:
lung_rna["patient"] = lung_rna["patient"].str.replace(".CEL","")

In [21]:
lung_rna.head(3)

,patient,RFC2,HSPA6,PAX8,GUCA1A,THRA,PTPN21,CCL5,CYP2E1,EPHB3,...,NAF1,CCDC63,C16orf52,LOC101929475,LOC389641,LOC400499,GALR3,NUS1P3,TMEM231,LOC100505915
0,GSM258551,7.427582,6.548161,7.511132,5.897988,6.352771,6.316551,9.066054,4.984092,7.723588,...,6.124568,4.456250,3.503017,4.176526,5.343218,6.784405,6.160561,4.188891,5.897904,5.936242
1,GSM258552,7.520136,7.640517,7.114491,5.362284,6.119202,7.606950,9.922118,6.259126,7.207981,...,6.082873,4.616373,3.677266,4.405394,4.757592,6.401263,5.551504,4.505053,5.065979,5.663643
2,GSM258553,8.607107,7.538416,7.631883,6.015103,6.567469,4.968846,9.886879,4.773658,7.958417,...,6.788019,5.006049,3.845536,4.448895,4.794517,6.048864,6.363465,4.626576,6.435507,5.151310


 -  Merge datasets

In [22]:
lung = lung_diag_sel.merge(lung_rna, how = "inner", on = "patient")

In [23]:
lung.head()

,patient,diagnosis,RFC2,HSPA6,PAX8,GUCA1A,THRA,PTPN21,CCL5,CYP2E1,...,NAF1,CCDC63,C16orf52,LOC101929475,LOC389641,LOC400499,GALR3,NUS1P3,TMEM231,LOC100505915
0,GSM257694,tumor,7.291018,6.101951,7.646472,5.947498,8.100123,7.523661,7.897441,5.371783,...,6.967920,5.033563,3.383837,4.343409,5.192321,6.780046,5.552982,4.536034,6.730379,5.380556
1,GSM257769,tumor,7.742699,6.793195,7.300662,6.043597,6.020626,5.422749,8.567400,5.445271,...,5.603514,4.655528,4.369526,4.087354,4.222128,5.665821,5.766637,4.559964,5.334571,5.359274
2,GSM257772,tumor,8.734882,7.858213,7.585445,5.357635,6.311921,6.130883,8.656558,4.500345,...,6.899072,4.761807,3.708143,4.029805,4.727088,6.147706,6.125370,4.442143,5.947681,5.060416
3,GSM257774,tumor,6.942824,7.888659,8.215977,6.049841,6.302013,6.860547,9.056366,4.939659,...,6.820519,4.811735,3.414262,4.788416,5.561117,6.429217,6.062942,3.909058,6.120516,6.597038
4,GSM257775,tumor,7.357759,6.469281,7.799088,5.556290,6.762244,7.071796,10.483605,6.620186,...,6.081121,5.046759,3.289144,4.512463,5.249973,6.387876,6.066651,4.062011,5.851772,5.493817


In [24]:
lung.shape

(1233, 20547)

In [25]:
lung["diagnosis"].value_counts()

tumor     1086
normal     147
Name: diagnosis, dtype: int64

# Preprocess datasets

 -  Check NAs

In [26]:
lung.isna().sum()

patient         0
diagnosis       0
RFC2            0
HSPA6           0
PAX8            0
               ..
LOC400499       0
GALR3           0
NUS1P3          0
TMEM231         0
LOC100505915    0
Length: 20547, dtype: int64

In [27]:
lung.isna().sum().sum()

0

 - Get statistics

In [28]:
description = lung.describe()

In [29]:
description

,RFC2,HSPA6,PAX8,GUCA1A,THRA,PTPN21,CCL5,CYP2E1,EPHB3,ESRRA,...,NAF1,CCDC63,C16orf52,LOC101929475,LOC389641,LOC400499,GALR3,NUS1P3,TMEM231,LOC100505915
count,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,...,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000
mean,7.862782,7.894766,7.553583,5.565346,6.755114,6.777950,8.879202,5.184229,7.672986,7.731847,...,6.696763,4.672221,3.670810,4.341304,4.813930,6.124879,5.977647,4.478923,6.397271,5.282103
std,0.619979,1.134942,0.310544,0.228870,0.491297,1.021887,1.276910,0.677038,0.662614,0.397457,...,0.457749,0.243894,0.363038,0.356266,0.278912,0.270033,0.277086,0.541623,0.882629,0.350889
min,6.276097,4.170535,6.654884,4.936375,4.940230,3.726160,3.279859,3.819645,6.027902,6.472335,...,4.931901,3.664662,2.747456,3.252498,4.053813,5.260215,5.016590,2.535423,3.663031,4.356603
25%,7.428962,7.212048,7.336212,5.404522,6.420790,6.122373,8.128056,4.761723,7.204518,7.473043,...,6.409584,4.516693,3.425392,4.144888,4.629651,5.945340,5.783148,4.118723,5.818150,5.047420
50%,7.787297,7.846989,7.533778,5.553631,6.693200,6.732411,9.020474,5.042403,7.586225,7.688224,...,6.661776,4.654898,3.628640,4.288275,4.795145,6.120696,5.976894,4.425481,6.332770,5.239218
75%,8.281218,8.523510,7.741773,5.711719,7.045156,7.494984,9.745018,5.435608,8.017562,7.951958,...,6.970940,4.815446,3.866297,4.448895,4.990243,6.284437,6.167232,4.792974,6.898128,5.490550
max,10.520540,13.064729,9.877611,6.816628,8.729741,11.224026,12.536486,10.389121,11.459393,9.837968,...,8.482028,5.614503,5.464491,7.276943,5.869484,7.738786,6.930588,6.949221,9.427133,6.954953


There are so many columns, so we cannot check it one by one, but the data seem to be already scaled so I'm going to check that by looking at the mean and the std of all these statistics

In [30]:
description_t = description.transpose()

In [31]:
description_t.head()

,count,mean,std,min,25%,50%,75%,max
RFC2,1233.0,7.862782,0.619979,6.276097,7.428962,7.787297,8.281218,10.520540
HSPA6,1233.0,7.894766,1.134942,4.170535,7.212048,7.846989,8.523510,13.064729
PAX8,1233.0,7.553583,0.310544,6.654884,7.336212,7.533778,7.741773,9.877611
GUCA1A,1233.0,5.565346,0.228870,4.936375,5.404522,5.553631,5.711719,6.816628
THRA,1233.0,6.755114,0.491297,4.940230,6.420790,6.693200,7.045156,8.729741


In [32]:
for i in description_t:
    print("mean of " + str(i) + ": " + str(np.mean(description_t[i])))
    print("std of " + str(i) + ": " + str(np.std(description_t[i])))
    print(" ")

mean of count: 1233.0
std of count: 0.0
 
mean of mean: 6.397032343375895
std of mean: 2.085211786168666
 
mean of std: 0.5573733740552759
std of std: 0.3467485675152558
 
mean of min: 4.655837810715979
std of min: 1.6747243239599405
 
mean of 25%: 6.032954253612461
std of 25%: 2.0252444137537493
 
mean of 50%: 6.356275401576593
std of 50%: 2.11851492124918
 
mean of 75%: 6.717023352376108
std of 75%: 2.204922907745834
 
mean of max: 8.953435765353149
std of max: 2.6354089232103
 


# Split and scale data

In [33]:
X = lung.drop(columns="diagnosis")
y = lung["diagnosis"]

In [34]:
X = X.set_index("patient")

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17, stratify=y)

In [36]:
train_X_original = X_train.copy()
test_X_original = X_test.copy()

 - Scaling

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # initiate transformer
X_train = scaler.fit_transform(X_train) # scale data
X_test = scaler.transform(X_test) # scale test set

In [38]:
X_train.shape

(986, 20545)

# Features reduction by PCA (0.9)

In [38]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.90)
X_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

In [39]:
X_reduced.shape

(986, 554)

In [40]:
X_test_reduced.shape

(247, 554)

# Models

### Libraries

*IMPORTANT*

The `precision` is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively **the ability of the classifier not to label as positive a sample that is negative**.

The `recall` is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively **the ability of the classifier to find all the positive samples**. Note that in binary classification, recall of the positive class is also known as `sensitivity`; recall of the negative class is `specificity`.

The most important for this work is to increase the **sensitivity** (to detect all cancer cases). Even though the other parameters are very important also.

In [40]:
# libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint
import time
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

### K Neighbors

#### Default

In [44]:
start_time = time.time()
neigh = KNeighborsClassifier()

neigh.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = neigh.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = neigh.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 0.9665314401622718
Test acc: 0.92


Train confussion matrix
[[109   2]
 [ 31 844]]


Test confussion matrix
[[ 33   3]
 [ 17 194]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.660000    0.984772  0.919028    0.822386      0.937436
recall      0.916667    0.919431  0.919028    0.918049      0.919028
f1-score    0.767442    0.950980  0.919028    0.859211      0.924230
support    36.000000  211.000000  0.919028  247.000000    247.000000


--- 1.584899663925171 seconds ---


#### Improved

In [45]:
start_time = time.time()
neigh1 = KNeighborsClassifier()

param_distribs={"n_neighbors": randint(low=3, high=30),
                "weights":["uniform", "distance"],
                "p":[1,2]}

neigh1_search = RandomizedSearchCV(neigh1,
                                   param_distribs,
                                   scoring="accuracy",
                                   n_iter=50,
                                   cv=3,
                                   n_jobs=-1,
                                   random_state=17)

neigh1_search.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = neigh1_search.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = neigh1_search.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 1.0
Test acc: 0.94


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 33   3]
 [ 13 198]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.717391    0.985075  0.935223    0.851233      0.946060
recall      0.916667    0.938389  0.935223    0.927528      0.935223
f1-score    0.804878    0.961165  0.935223    0.883022      0.938386
support    36.000000  211.000000  0.935223  247.000000    247.000000


--- 16.292540788650513 seconds ---


### Random Forest

#### Default

In [46]:
start_time = time.time()
randomfor = RandomForestClassifier()

randomfor.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = randomfor.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = randomfor.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 1.0
Test acc: 0.9


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 11  25]
 [  0 211]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   1.000000    0.894068  0.898785    0.947034      0.909507
recall      0.305556    1.000000  0.898785    0.652778      0.898785
f1-score    0.468085    0.944072  0.898785    0.706078      0.874697
support    36.000000  211.000000  0.898785  247.000000    247.000000


--- 1.1291840076446533 seconds ---


#### "Improved"

In [47]:
start_time = time.time()

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['None', 'sqrt', "log2"]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

randomfor_search1 = RandomizedSearchCV(RandomForestClassifier(),
                                   random_grid,
                                   scoring="accuracy",
                                   n_iter=8,
                                   cv=3,
                                   n_jobs=-1,
                                   random_state=17,
                                   verbose=2)



randomfor_search1.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = randomfor_search1.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = randomfor_search1.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   29.7s finished


Train acc: 1.0
Test acc: 0.9


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 13  23]
 [  1 210]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.928571    0.901288  0.902834    0.914929      0.905264
recall      0.361111    0.995261  0.902834    0.678186      0.902834
f1-score    0.520000    0.945946  0.902834    0.732973      0.883865
support    36.000000  211.000000  0.902834  247.000000    247.000000


--- 32.735334157943726 seconds ---


In [146]:
start_time = time.time()

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['None', 'sqrt', "log2"]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

randomfor_search2 = RandomizedSearchCV(RandomForestClassifier(),
                                   random_grid,
                                   scoring="accuracy",
                                   n_iter=200,
                                   cv=3,
                                   n_jobs=3,
                                   random_state=17,
                                   verbose=5)



randomfor_search2.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = randomfor_search2.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = randomfor_search2.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   53.4s
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  2.6min
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:  6.0min
[Parallel(n_jobs=3)]: Done 282 tasks      | elapsed: 10.2min
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed: 17.9min
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed: 24.8min finished


Train acc: 1.0
Test acc: 0.9


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 12  24]
 [  1 210]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.923077    0.897436  0.898785    0.910256      0.901173
recall      0.333333    0.995261  0.898785    0.664297      0.898785
f1-score    0.489796    0.943820  0.898785    0.716808      0.877647
support    36.000000  211.000000  0.898785  247.000000    247.000000


--- 1498.3766441345215 seconds ---


### Support Vector Machine (SVM)

#### Default

In [48]:
start_time = time.time()
svc = SVC()

svc.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = svc.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = svc.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 0.9908722109533469
Test acc: 0.96


Train confussion matrix
[[106   5]
 [  4 871]]


Test confussion matrix
[[ 30   6]
 [  4 207]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.882353    0.971831  0.959514    0.927092      0.958790
recall      0.833333    0.981043  0.959514    0.907188      0.959514
f1-score    0.857143    0.976415  0.959514    0.916779      0.959031
support    36.000000  211.000000  0.959514  247.000000    247.000000


--- 0.4478340148925781 seconds ---


In [119]:
pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True))

,normal,tumor,accuracy,macro avg,weighted avg
precision,0.882353,0.971831,0.959514,0.927092,0.958790
recall,0.833333,0.981043,0.959514,0.907188,0.959514
f1-score,0.857143,0.976415,0.959514,0.916779,0.959031
support,36.000000,211.000000,0.959514,247.000000,247.000000


### Logistic

#### Default

In [49]:
start_time = time.time()
logreg = LogisticRegression()

logreg.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = logreg.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = logreg.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 1.0
Test acc: 0.98


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 32   4]
 [  2 209]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.941176    0.981221  0.975709    0.961199      0.975384
recall      0.888889    0.990521  0.975709    0.939705      0.975709
f1-score    0.914286    0.985849  0.975709    0.950067      0.975419
support    36.000000  211.000000  0.975709  247.000000    247.000000


--- 0.1702878475189209 seconds ---


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### "Improved" (fail)

In [50]:
start_time = time.time()
logreg = LogisticRegression()


grid_values = {'penalty': ['l2'],
               'C':[1],
              }

logreg_search1 = GridSearchCV(logreg,
                            grid_values,
                            scoring="accuracy",  
                            cv=10
                            )

logreg_search1.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = logreg_search1.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = logreg_search1.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

Train acc: 1.0
Test acc: 0.98


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 32   4]
 [  2 209]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.941176    0.981221  0.975709    0.961199      0.975384
recall      0.888889    0.990521  0.975709    0.939705      0.975709
f1-score    0.914286    0.985849  0.975709    0.950067      0.975419
support    36.000000  211.000000  0.975709  247.000000    247.000000


--- 0.8021800518035889 seconds ---


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [51]:
logreg_search1.best_params_

{'C': 1, 'penalty': 'l2'}

### Gradient Boosting

In [58]:
start_time = time.time()
gradboost = GradientBoostingClassifier()

gradboost.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = gradboost.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = gradboost.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 1.0
Test acc: 0.93


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 23  13]
 [  4 207]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.851852    0.940909  0.931174    0.896380      0.927929
recall      0.638889    0.981043  0.931174    0.809966      0.931174
f1-score    0.730159    0.960557  0.931174    0.845358      0.926977
support    36.000000  211.000000  0.931174  247.000000    247.000000


--- 13.93536114692688 seconds ---


### Ada Boosting

In [59]:
start_time = time.time()
adaboost = AdaBoostClassifier()

adaboost.fit(X_reduced, y_train)

# Check accuracy in the testset
y_predict = adaboost.predict(X_reduced)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = adaboost.predict(X_test_reduced)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 1.0
Test acc: 0.94


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 27   9]
 [  6 205]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.818182    0.957944  0.939271    0.888063      0.937574
recall      0.750000    0.971564  0.939271    0.860782      0.939271
f1-score    0.782609    0.964706  0.939271    0.873657      0.938165
support    36.000000  211.000000  0.939271  247.000000    247.000000


--- 2.951756000518799 seconds ---


# Models without PCA-reduction

### Support Vector Machine (SVM)

In [52]:
start_time = time.time()
svc = SVC()

svc.fit(X_train, y_train)

# Check accuracy in the testset
y_predict = svc.predict(X_train)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = svc.predict(X_test)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 0.9959432048681541
Test acc: 0.96


Train confussion matrix
[[108   3]
 [  1 874]]


Test confussion matrix
[[ 28   8]
 [  2 209]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.933333    0.963134  0.959514    0.948233      0.958790
recall      0.777778    0.990521  0.959514    0.884150      0.959514
f1-score    0.848485    0.976636  0.959514    0.912560      0.957958
support    36.000000  211.000000  0.959514  247.000000    247.000000


--- 19.097121953964233 seconds ---


### Logistic regression

In [53]:
start_time = time.time()
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

# Check accuracy in the testset
y_predict = logreg.predict(X_train)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = logreg.predict(X_test)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 1.0
Test acc: 0.92


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 24  12]
 [  7 204]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.774194    0.944444  0.923077    0.859319      0.919631
recall      0.666667    0.966825  0.923077    0.816746      0.923077
f1-score    0.716418    0.955504  0.923077    0.835961      0.920657
support    36.000000  211.000000  0.923077  247.000000    247.000000


--- 2.8998048305511475 seconds ---


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Ada boosting

In [68]:
start_time = time.time()
adaboost = AdaBoostClassifier()

adaboost.fit(X_train, y_train)

# Check accuracy in the testset
y_predict = adaboost.predict(X_train)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = adaboost.predict(X_test)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 1.0
Test acc: 0.98


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 32   4]
 [  2 209]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.941176    0.981221  0.975709    0.961199      0.975384
recall      0.888889    0.990521  0.975709    0.939705      0.975709
f1-score    0.914286    0.985849  0.975709    0.950067      0.975419
support    36.000000  211.000000  0.975709  247.000000    247.000000


--- 101.90455222129822 seconds ---


#### Improved

In [80]:
start_time = time.time()
adaboost = AdaBoostClassifier()

param_distribs={"n_estimators": np.arange(1, 500, 5),
                "learning_rate":[0.1],
                #"learning_rate":np.arange(0.1, 200, 1),
               "random_state":[17]}

ada_search1 = RandomizedSearchCV(adaboost,
                                   param_distribs,
                                   scoring="accuracy",
                                   n_iter=4,
                                   cv=3,
                                   n_jobs=-1,
                                   random_state=17,
                                   verbose=3)

ada_search1.fit(X_train, y_train)

# Check accuracy in the testset
y_predict = ada_search1.predict(X_train)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = ada_search1.predict(X_test)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed: 19.8min remaining:  4.0min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 19.8min finished


Train acc: 1.0
Test acc: 0.98


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 32   4]
 [  2 209]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.941176    0.981221  0.975709    0.961199      0.975384
recall      0.888889    0.990521  0.975709    0.939705      0.975709
f1-score    0.914286    0.985849  0.975709    0.950067      0.975419
support    36.000000  211.000000  0.975709  247.000000    247.000000


--- 1970.1237869262695 seconds ---


In [81]:
ada_search1.best_params_

{'random_state': 17, 'n_estimators': 356, 'learning_rate': 0.1}

In [41]:
start_time = time.time()
adaboost = AdaBoostClassifier()

param_distribs={"n_estimators": [356],
                "learning_rate":[1], #default = 1
               "random_state":[17]}

ada_search1 = RandomizedSearchCV(adaboost,
                                   param_distribs,
                                   scoring="accuracy",
                                   n_iter=1,
                                   cv=3,
                                   n_jobs=-1,
                                   random_state=17,
                                   verbose=3)

ada_search1.fit(X_train, y_train)

# Check accuracy in the testset
y_predict = ada_search1.predict(X_train)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = ada_search1.predict(X_test)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 15.0min finished


Train acc: 1.0
Test acc: 0.99


Train confussion matrix
[[118   0]
 [  0 868]]


Test confussion matrix
[[ 28   1]
 [  1 217]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.965517    0.995413  0.991903    0.980465      0.991903
recall      0.965517    0.995413  0.991903    0.980465      0.991903
f1-score    0.965517    0.995413  0.991903    0.980465      0.991903
support    29.000000  218.000000  0.991903  247.000000    247.000000


--- 1699.4455180168152 seconds ---


In [43]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, y_test_predict)
mat_df = pd.DataFrame(mat)
mat_df = mat_df.rename({0:"No Cancer", 1: "Cancer"})
mat_df = mat_df.rename(columns={0:"No Cancer", 1: "Cancer"})
mat_df["total"] = mat_df["No Cancer"] + mat_df["Cancer"]
mat_df = mat_df.append(mat_df.sum(axis=0).rename('total'))
mat_df.index.names = ['Real']
columns=[('Predicted', 'No Cancer'), ('Predicted', 'Cancer'), ('Predicted', 'total')]
mat_df.columns = pd.MultiIndex.from_tuples(columns)
mat_df

Predicted             
          No Cancer Cancer total
Real                            
No Cancer        28      1    29
Cancer            1    217   218
total            29    218   247

In [47]:
precision =pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)).iloc[0:1,0:2]
#precision = precision["Low"],precision["Medium"]
precision = pd.DataFrame(precision)
precision.columns= ["No Cancer","Cancer"]
precision

,No Cancer,Cancer
precision,0.965517,0.995413


In [89]:
ada_search1.best_params_

{'random_state': 17, 'n_estimators': 356, 'learning_rate': 1}

# Models with different values of PCA reduction

### Testing loop for best PCA variance

In [190]:
variance = np.arange(0.05,1,0.05)
variance

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

In [191]:
for v in variance:
    pca_2 = PCA(n_components=v)
    X_reduced_2 = pca_2.fit_transform(X_train)
    X_test_reduced_2 = pca_2.transform(X_test)
    
    start_time = time.time()
    logreg = LogisticRegression()

    logreg.fit(X_reduced_2, y_train)

    # Check accuracy in the testset
    y_predict = logreg.predict(X_reduced_2)
    print("Train acc: " + str(accuracy_score(y_train, y_predict)))

    # Check accuracy in the testset
    y_test_predict = logreg.predict(X_test_reduced_2)
    print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

    print("\n")
    print("Train confussion matrix")
    print(confusion_matrix(y_train, y_predict))

    print("\n")
    print("Test confussion matrix")
    print(confusion_matrix(y_test, y_test_predict))

    print("\n")
    print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

    print("\n")
    print("--- %s seconds ---" % (time.time() - start_time))
    print("---------------------------------------------------------")
    print("\n")


Train acc: 0.9219066937119675
Test acc: 0.88


Train confussion matrix
[[ 54  57]
 [ 20 855]]


Test confussion matrix
[[ 13  23]
 [  6 205]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.684211    0.899123  0.882591    0.791667      0.867800
recall      0.361111    0.971564  0.882591    0.666338      0.882591
f1-score    0.472727    0.933941  0.882591    0.703334      0.866719
support    36.000000  211.000000  0.882591  247.000000    247.000000


--- 0.02686285972595215 seconds ---
---------------------------------------------------------


Train acc: 0.9239350912778904
Test acc: 0.9


Train confussion matrix
[[ 65  46]
 [ 29 846]]


Test confussion matrix
[[ 17  19]
 [  5 206]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.772727    0.915556  0.902834    0.844141      0.894738
recall      0.472222    0.976303  0.902834    0.724263      0.902834
f1-score    0.586207    0.944954  0.902834    0.765581      0.8

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Train acc: 0.9787018255578094
Test acc: 0.95


Train confussion matrix
[[100  11]
 [ 10 865]]


Test confussion matrix
[[ 29   7]
 [  5 206]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.852941    0.967136  0.951417    0.910039      0.950492
recall      0.805556    0.976303  0.951417    0.890929      0.951417
f1-score    0.828571    0.971698  0.951417    0.900135      0.950838
support    36.000000  211.000000  0.951417  247.000000    247.000000


--- 0.048111915588378906 seconds ---
---------------------------------------------------------




/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Train acc: 0.9989858012170385
Test acc: 0.94


Train confussion matrix
[[111   0]
 [  1 874]]


Test confussion matrix
[[ 27   9]
 [  6 205]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.818182    0.957944  0.939271    0.888063      0.937574
recall      0.750000    0.971564  0.939271    0.860782      0.939271
f1-score    0.782609    0.964706  0.939271    0.873657      0.938165
support    36.000000  211.000000  0.939271  247.000000    247.000000


--- 0.0514981746673584 seconds ---
---------------------------------------------------------




...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Train acc: 1.0
Test acc: 0.98


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 32   4]
 [  1 210]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.969697    0.981308  0.979757    0.975503      0.979616
recall      0.888889    0.995261  0.979757    0.942075      0.979757
f1-score    0.927536    0.988235  0.979757    0.957886      0.979388
support    36.000000  211.000000  0.979757  247.000000    247.000000


--- 0.08190512657165527 seconds ---
---------------------------------------------------------




/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Train acc: 1.0
Test acc: 0.98


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 32   4]
 [  2 209]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.941176    0.981221  0.975709    0.961199      0.975384
recall      0.888889    0.990521  0.975709    0.939705      0.975709
f1-score    0.914286    0.985849  0.975709    0.950067      0.975419
support    36.000000  211.000000  0.975709  247.000000    247.000000


--- 0.11392807960510254 seconds ---
---------------------------------------------------------


Train acc: 1.0
Test acc: 0.97


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 31   5]
 [  3 208]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.911765    0.976526  0.967611    0.944145      0.967087
recall      0.861111    0.985782  0.967611    0.923447      0.967611
f1-score    0.885714    0.981132  0.967611    0.933423      0.967225
support    36.000000  2

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [87]:
# We keep 0.85 as the best one
pca_3 = PCA(n_components=0.85)
X_reduced_3 = pca_3.fit_transform(X_train)
X_test_reduced_3 = pca_3.transform(X_test)

### Support Vector Machine (SVM) with PCA (0.85)

In [55]:
start_time = time.time()
svc = SVC()

svc.fit(X_reduced_3, y_train)

# Check accuracy in the testset
y_predict = svc.predict(X_reduced_3)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = svc.predict(X_test_reduced_3)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 0.9888438133874239
Test acc: 0.96


Train confussion matrix
[[105   6]
 [  5 870]]


Test confussion matrix
[[ 30   6]
 [  4 207]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.882353    0.971831  0.959514    0.927092      0.958790
recall      0.833333    0.981043  0.959514    0.907188      0.959514
f1-score    0.857143    0.976415  0.959514    0.916779      0.959031
support    36.000000  211.000000  0.959514  247.000000    247.000000


--- 0.3410048484802246 seconds ---


### Logistic regression with PCA (0.85)


In [88]:
start_time = time.time()
logreg = LogisticRegression()

logreg.fit(X_reduced_3, y_train)

# Check accuracy in the trainset
y_predict = logreg.predict(X_reduced_3)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = logreg.predict(X_test_reduced_3)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 1.0
Test acc: 0.99


Train confussion matrix
[[118   0]
 [  0 868]]


Test confussion matrix
[[ 27   2]
 [  1 217]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.964286    0.990868  0.987854    0.977577      0.987747
recall      0.931034    0.995413  0.987854    0.963224      0.987854
f1-score    0.947368    0.993135  0.987854    0.970252      0.987762
support    29.000000  218.000000  0.987854  247.000000    247.000000


--- 0.13643431663513184 seconds ---


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### "Improved" (fail)

In [67]:
start_time = time.time()
logreg = LogisticRegression()


grid_values = {
               'C':[0.0001,0.001,0.01,0.1,1,10,100,1000,10000,100000],
              }

logreg_search2 = GridSearchCV(logreg,
                            grid_values,
                            scoring="accuracy",  
                            cv=10
                            )

logreg_search2.fit(X_reduced_3, y_train)

# Check accuracy in the testset
y_predict = logreg_search2.predict(X_reduced_3)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = logreg_search2.predict(X_test_reduced_3)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

Train acc: 1.0
Test acc: 0.97


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 31   5]
 [  2 209]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.939394    0.976636   0.97166    0.958015      0.971208
recall      0.861111    0.990521   0.97166    0.925816      0.971660
f1-score    0.898551    0.983529   0.97166    0.941040      0.971144
support    36.000000  211.000000   0.97166  247.000000    247.000000


--- 5.053434133529663 seconds ---


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

### Ada Boosting with PCA (0.85)

In [74]:
start_time = time.time()
adaboost = AdaBoostClassifier()

param_distribs={"n_estimators": np.arange(1, 500, 5),
                #"learning_rate":np.arange(0.1, 200, 1),
               "random_state":[17]}

ada_search1 = RandomizedSearchCV(adaboost,
                                   param_distribs,
                                   scoring="accuracy",
                                   n_iter=3,
                                   cv=3,
                                   n_jobs=-1,
                                   random_state=17,
                                   verbose=3)


ada_search1.fit(X_reduced_3, y_train)

# Check accuracy in the testset
y_predict = ada_search1.predict(X_reduced_3)
print("Train acc: " + str(accuracy_score(y_train, y_predict)))

# Check accuracy in the testset
y_test_predict = ada_search1.predict(X_test_reduced_3)
print("Test acc: " + str(round(accuracy_score(y_test, y_test_predict),2)))

print("\n")
print("Train confussion matrix")
print(confusion_matrix(y_train, y_predict))

print("\n")
print("Test confussion matrix")
print(confusion_matrix(y_test, y_test_predict))

print("\n")
print(pd.DataFrame(classification_report(y_test, y_test_predict, output_dict=True)))

print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:   26.1s remaining:   13.1s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:   26.8s finished


Train acc: 1.0
Test acc: 0.97


Train confussion matrix
[[111   0]
 [  0 875]]


Test confussion matrix
[[ 33   3]
 [  5 206]]


              normal       tumor  accuracy   macro avg  weighted avg
precision   0.868421    0.985646  0.967611    0.927033      0.968561
recall      0.916667    0.976303  0.967611    0.946485      0.967611
f1-score    0.891892    0.980952  0.967611    0.936422      0.967972
support    36.000000  211.000000  0.967611  247.000000    247.000000


--- 43.04130506515503 seconds ---


In [75]:
ada_search1.best_params_

{'random_state': 17, 'n_estimators': 356}